# Case Study 3 : Textual analysis of movie reviews

** Due Date: November 17, 2016 5:59PM**

*------------

<img src="http://www.conversational-technologies.com/nldemos/nlWordle.GIF">

**TEAM Members:** Please EDIT this cell and add the names of all the team members in your team

    member 1
    
    member 2
    
    ...

**Desired outcome of the case study.**
* In this case study we will look at movie reviews from the v2.0 polarity dataset comes from
the http://www.cs.cornell.edu/people/pabo/movie-review-data.
    * It contains written reviews of movies divided into positive and negative reviews.
* As in Case Study 2 idea is to *analyze* the data set, make *conjectures*, support or refute those conjectures with *data*, and *tell a story* about the data!
    
**Required Readings:** 
* This case study will be based upon the scikit-learn Python library
* We will build upon the turtorial "Working With Text Data" which can be found at http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

**Case study assumptions:**
* You have access to a python installation

**Required Python libraries:**
* Numpy (www.numpy.org) (should already be installed from Case Study 2)
* Matplotlib (matplotlib.org) (should already be installed from Case Study 2)
* Scikit-learn (scikit-learn.org) (avaiable from Enthought Canopy)
* You are also welcome to use the Python Natural Language Processing Toolkit (www.nltk.org) (though it is not required).

** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

## Problem 1 (20 points): Complete Exercise 2: Sentiment Analysis on movie reviews from http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

* Assuming that you have downloaded the scikit-learn source code:
    * The data cane be downloaded using doc/tutorial/text_analytics/data/movie_reviews/fetch_data.py
    * A skeleton for the solution can be found in doc/tutorial/text_analytics/skeletons/exercise_02_sentiment.py
    * A completed solution can be found in doc/tutorial/text_analytics/solutions/exercise_02_sentiment.py
* **It is ok to use the solution provided in the scikit-learn distribution as a starting place for your work.**

### Modify the solution to Exercise 2 so that it can run in this iPython notebook
* This will likely involved moving around data files and/or small modifications to the script.

In [1]:
# import libs
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
# read in data
dataset = load_files('./text_analytics/data/movie_reviews/txt_sentoken/', shuffle=False)
print("n_samples: %d" % len(dataset.data))

n_samples: 2000


In [3]:
# split into training and test data sets
docs_train, docs_test, y_train, y_test = train_test_split(dataset.data, dataset.target, 
                                                          test_size=0.25, random_state=None)

In [4]:
# building pipeline which is used to process the data in order
pipeline = Pipeline([
        ('vect', TfidfVectorizer(min_df=3, max_df=0.95)), # vectorization
        ('clf', LinearSVC(C=1000)), # linear support vector classifier
    ])

In [5]:
# set up parameters for voctorization process and use grid_search to fit the training data
parameters = {
        'vect__ngram_range': [(1, 1), (1, 2)],
    }
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1)
grid_search.fit(docs_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.95, max_features=None, min_df=3,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': [(1, 1), (1, 2)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [6]:
# print out model score
n_candidates = len(grid_search.cv_results_['params'])
for i in range(n_candidates):
    print(i, 'params - %s; mean - %0.2f; std - %0.2f'
             % (grid_search.cv_results_['params'][i],
                grid_search.cv_results_['mean_test_score'][i],
                grid_search.cv_results_['std_test_score'][i]))


0 params - {'vect__ngram_range': (1, 1)}; mean - 0.82; std - 0.01
1 params - {'vect__ngram_range': (1, 2)}; mean - 0.84; std - 0.00


In [7]:
# predict the test data set and print out the result with best cross validation model
y_predicted = grid_search.predict(docs_test)
print(metrics.classification_report(y_test, y_predicted, 
                                    target_names=dataset.target_names)) # classification result
cm = metrics.confusion_matrix(y_test, y_predicted) # confusion matrix
print(cm)

             precision    recall  f1-score   support

        neg       0.88      0.86      0.87       252
        pos       0.86      0.88      0.87       248

avg / total       0.87      0.87      0.87       500

[[217  35]
 [ 31 217]]


## Problem 2 (20 points): Explore the scikit-learn TfidVectorizer class

**Read the documentation for the TfidVectorizer class at http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html.** 
* Define the term frequency–inverse document frequency (TF-IDF) statistic (http://en.wikipedia.org/wiki/Tf%E2%80%93idf will likely help).
* Run the TfidVectorizer class on the training data above (docs_train).
* Explore the min_df and max_df parameters of TfidVectorizer.  What do they mean? How do they change the features you get?
* Explore the ngram_range parameter of TfidVectorizer.  What does it mean? How does it change the features you get? (Note, large values  of ngram_range may take a long time to run!)

In [8]:
import numpy as np

In [9]:
for i in np.arange(0.05, 0.3, 0.05):
    vector = TfidfVectorizer(min_df=i, max_df=0.9)
    matrix = vector.fit_transform(docs_train)
    print ('Current min_df is %f and max_df is %f. The number of nonzero elements in matrix are %d' % 
           (i, 0.9, matrix.nnz))

Current min_df is 0.050000 and max_df is 0.900000. The number of nonzero elements in matrix are 272183
Current min_df is 0.100000 and max_df is 0.900000. The number of nonzero elements in matrix are 210586
Current min_df is 0.150000 and max_df is 0.900000. The number of nonzero elements in matrix are 176235
Current min_df is 0.200000 and max_df is 0.900000. The number of nonzero elements in matrix are 151707
Current min_df is 0.250000 and max_df is 0.900000. The number of nonzero elements in matrix are 133792


In [10]:
for i in np.arange(0.95, 0.7, -0.05):
    vector = TfidfVectorizer(min_df=0.1, max_df=i)
    matrix = vector.fit_transform(docs_train)
    print ('Current min_df is %f and max_df is %f. The number of nonzero elements in matrix are %d' % 
           (0.1, i, matrix.nnz))

Current min_df is 0.100000 and max_df is 0.950000. The number of nonzero elements in matrix are 213400
Current min_df is 0.100000 and max_df is 0.900000. The number of nonzero elements in matrix are 210586
Current min_df is 0.100000 and max_df is 0.850000. The number of nonzero elements in matrix are 198676
Current min_df is 0.100000 and max_df is 0.800000. The number of nonzero elements in matrix are 191232
Current min_df is 0.100000 and max_df is 0.750000. The number of nonzero elements in matrix are 185535


In [11]:
ngram_candidate = [(1,1), (1,2), (1,3)]
for ngram in ngram_candidate:
    vector = TfidfVectorizer(min_df=0.1, max_df=0.9, ngram_range=ngram)
    matrix = vector.fit_transform(docs_train)
    print ('Current ngram range is (%d,%d). The number of nonzero elements in matrix are %d' % 
           (ngram[0], ngram[1], matrix.nnz))

Current ngram range is (1,1). The number of nonzero elements in matrix are 210586
Current ngram range is (1,2). The number of nonzero elements in matrix are 279160
Current ngram range is (1,3). The number of nonzero elements in matrix are 283401


*------------------------

## Problem 3 (20 points): Machine learning algorithms


* Based upon Problem 2 pick some parameters for TfidfVectorizer
    * "fit" your TfidfVectorizer using docs_train
    * Compute "Xtrain", a Tf-idf-weighted document-term matrix using the transform function on docs_train
    * Compute "Xtest", a Tf-idf-weighted document-term matrix using the transform function on docs_test
    * Note, be sure to use the same Tf-idf-weighted class (**"fit" using docs_train**) to transform **both** docs_test and docs_train
* Examine two classifiers provided by scikit-learn 
    * LinearSVC
    * KNeighborsClassifier
    * Try a number of different parameter settings for each and judge your performance using a confusion matrix (see Problem 1 for an example).
* Does one classifier, or one set of parameters work better?
    * Why do you think it might be working better?
* For a particular choice of parameters and classifier, look at 2 examples where the prediction was incorrect.
    * Can you conjecture on why the classifier made a mistake for this prediction?

In [27]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
# prepare the Xtrain and Xtest
vector = TfidfVectorizer(min_df=3, max_df=0.95, ngram_range=(1,2))
vector.fit(docs_train)

Xtrain = vector.transform(docs_train)
Xtest  = vector.transform(docs_test)

In [16]:
# examine LinearSVC
svc_C = [1000, 900, 800, 700, 600, 500, 400, 300, 200]

for param in svc_C:
    svc = LinearSVC(C=param)
    svc.fit(Xtrain, y_train)
    y_predicted = svc.predict(Xtest)
    
    print ('Current parameters of LinearSVC are C=%d' % param)
    cm = metrics.confusion_matrix(y_test, y_predicted) # confusion matrix
    print ('Final confusion matrix is:')
    print (cm)
    print ('\r')

Current parameters of LinearSVC are C=1000
Final confusion matrix is:
[[217  35]
 [ 31 217]]

Current parameters of LinearSVC are C=900
Final confusion matrix is:
[[217  35]
 [ 31 217]]

Current parameters of LinearSVC are C=800
Final confusion matrix is:
[[217  35]
 [ 31 217]]

Current parameters of LinearSVC are C=700
Final confusion matrix is:
[[217  35]
 [ 31 217]]

Current parameters of LinearSVC are C=600
Final confusion matrix is:
[[217  35]
 [ 31 217]]

Current parameters of LinearSVC are C=500
Final confusion matrix is:
[[217  35]
 [ 31 217]]

Current parameters of LinearSVC are C=400
Final confusion matrix is:
[[217  35]
 [ 31 217]]

Current parameters of LinearSVC are C=300
Final confusion matrix is:
[[217  35]
 [ 31 217]]

Current parameters of LinearSVC are C=200
Final confusion matrix is:
[[217  35]
 [ 31 217]]



In [17]:
# examine LinearSVC
KNN_n_neighbors = [1, 2, 3, 4, 5, 6, 7, 8, 9]

for param in KNN_n_neighbors:
    KNN = KNeighborsClassifier(n_neighbors=param)
    KNN.fit(Xtrain, y_train)
    y_predicted = KNN.predict(Xtest)
    
    print ('Current parameters of KNN classifier are n_neighbors=%d' % param)
    cm = metrics.confusion_matrix(y_test, y_predicted) # confusion matrix
    print ('Final confusion matrix is:')
    print (cm)
    print ('\r')

Current parameters of KNN classifier are n_neighbors=1
Final confusion matrix is:
[[162  90]
 [ 74 174]]

Current parameters of KNN classifier are n_neighbors=2
Final confusion matrix is:
[[193  59]
 [124 124]]

Current parameters of KNN classifier are n_neighbors=3
Final confusion matrix is:
[[150 102]
 [ 65 183]]

Current parameters of KNN classifier are n_neighbors=4
Final confusion matrix is:
[[181  71]
 [ 93 155]]

Current parameters of KNN classifier are n_neighbors=5
Final confusion matrix is:
[[135 117]
 [ 50 198]]

Current parameters of KNN classifier are n_neighbors=6
Final confusion matrix is:
[[161  91]
 [ 67 181]]

Current parameters of KNN classifier are n_neighbors=7
Final confusion matrix is:
[[146 106]
 [ 45 203]]

Current parameters of KNN classifier are n_neighbors=8
Final confusion matrix is:
[[163  89]
 [ 61 187]]

Current parameters of KNN classifier are n_neighbors=9
Final confusion matrix is:
[[142 110]
 [ 45 203]]



In [18]:
# LinearSVC
print ('======= Start Evaluation of LinearSVC =======')
pipeline = Pipeline([
        ('vect', TfidfVectorizer()), # vectorization
        ('clf', LinearSVC()), # linear support vector classifier
    ])

parameters = {
        'vect__ngram_range': [(1, 2), (1, 3)],
        'vect__min_df': [2, 3, 4],
        'vect__max_df': [0.98, 0.95, 0.92],
        'clf__C': [1000, 800, 600],
    }

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1)
grid_search.fit(docs_train, y_train)

n_candidates = len(grid_search.cv_results_['params'])
for i in range(n_candidates):
    print(i, 'params - %s; mean - %0.2f; std - %0.2f'
             % (grid_search.cv_results_['params'][i],
                grid_search.cv_results_['mean_test_score'][i],
                grid_search.cv_results_['std_test_score'][i]))

y_predicted = grid_search.predict(docs_test)
print(metrics.classification_report(y_test, y_predicted, 
                                    target_names=dataset.target_names)) # classification result
cm = metrics.confusion_matrix(y_test, y_predicted) # confusion matrix
print(cm)

======= Start Evaluation of LinearSVC =======
0 params - {'vect__ngram_range': (1, 2), 'clf__C': 1000, 'vect__min_df': 2, 'vect__max_df': 0.98}; mean - 0.84; std - 0.01
1 params - {'vect__ngram_range': (1, 3), 'clf__C': 1000, 'vect__min_df': 2, 'vect__max_df': 0.98}; mean - 0.84; std - 0.01
2 params - {'vect__ngram_range': (1, 2), 'clf__C': 1000, 'vect__min_df': 3, 'vect__max_df': 0.98}; mean - 0.84; std - 0.01
3 params - {'vect__ngram_range': (1, 3), 'clf__C': 1000, 'vect__min_df': 3, 'vect__max_df': 0.98}; mean - 0.84; std - 0.01
4 params - {'vect__ngram_range': (1, 2), 'clf__C': 1000, 'vect__min_df': 4, 'vect__max_df': 0.98}; mean - 0.84; std - 0.01
5 params - {'vect__ngram_range': (1, 3), 'clf__C': 1000, 'vect__min_df': 4, 'vect__max_df': 0.98}; mean - 0.84; std - 0.01
6 params - {'vect__ngram_range': (1, 2), 'clf__C': 1000, 'vect__min_df': 2, 'vect__max_df': 0.95}; mean - 0.84; std - 0.01
7 params - {'vect__ngram_range': (1, 3), 'clf__C': 1000, 'vect__min_df': 2, 'vect__max_df': 0

In [19]:
# K nearst neighbor
print ('======= Start Evaluation of KNN =======')
pipeline = Pipeline([
        ('vect', TfidfVectorizer()), # vectorization
        ('clf', KNeighborsClassifier()), # KNN classifier
    ])

parameters = {
        'vect__ngram_range': [(1, 2), (1, 3)],
        'vect__min_df': [2, 3, 4],
        'vect__max_df': [0.98, 0.95, 0.92],
        'clf__n_neighbors': [2, 4, 6],
    }

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1)
grid_search.fit(docs_train, y_train)

n_candidates = len(grid_search.cv_results_['params'])
for i in range(n_candidates):
    print(i, 'params - %s; mean - %0.2f; std - %0.2f'
             % (grid_search.cv_results_['params'][i],
                grid_search.cv_results_['mean_test_score'][i],
                grid_search.cv_results_['std_test_score'][i]))

y_predicted = grid_search.predict(docs_test)
print(metrics.classification_report(y_test, y_predicted, 
                                    target_names=dataset.target_names)) # classification result
cm = metrics.confusion_matrix(y_test, y_predicted) # confusion matrix
print(cm)

======= Start Evaluation of KNN =======
0 params - {'clf__n_neighbors': 2, 'vect__ngram_range': (1, 2), 'vect__min_df': 2, 'vect__max_df': 0.98}; mean - 0.64; std - 0.02
1 params - {'clf__n_neighbors': 2, 'vect__ngram_range': (1, 3), 'vect__min_df': 2, 'vect__max_df': 0.98}; mean - 0.64; std - 0.01
2 params - {'clf__n_neighbors': 2, 'vect__ngram_range': (1, 2), 'vect__min_df': 3, 'vect__max_df': 0.98}; mean - 0.65; std - 0.02
3 params - {'clf__n_neighbors': 2, 'vect__ngram_range': (1, 3), 'vect__min_df': 3, 'vect__max_df': 0.98}; mean - 0.65; std - 0.02
4 params - {'clf__n_neighbors': 2, 'vect__ngram_range': (1, 2), 'vect__min_df': 4, 'vect__max_df': 0.98}; mean - 0.65; std - 0.02
5 params - {'clf__n_neighbors': 2, 'vect__ngram_range': (1, 3), 'vect__min_df': 4, 'vect__max_df': 0.98}; mean - 0.66; std - 0.02
6 params - {'clf__n_neighbors': 2, 'vect__ngram_range': (1, 2), 'vect__min_df': 2, 'vect__max_df': 0.95}; mean - 0.64; std - 0.02
7 params - {'clf__n_neighbors': 2, 'vect__ngram_ra

In [23]:
# Random Forest
print ('======= Start Evaluation of Random Forest =======')
pipeline = Pipeline([
        ('vect', TfidfVectorizer()), # vectorization
        ('clf', RandomForestClassifier()), # random forest classifier
    ])

parameters = {
        'vect__ngram_range': [(1, 2), (1, 3)],
        'vect__min_df': [2, 3, 4],
        'vect__max_df': [0.98, 0.95, 0.92],
        'clf__n_estimators': [4, 8, 12],
    }

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1)
grid_search.fit(docs_train, y_train)

n_candidates = len(grid_search.cv_results_['params'])
for i in range(n_candidates):
    print(i, 'params - %s; mean - %0.2f; std - %0.2f'
             % (grid_search.cv_results_['params'][i],
                grid_search.cv_results_['mean_test_score'][i],
                grid_search.cv_results_['std_test_score'][i]))

y_predicted = grid_search.predict(docs_test)
print(metrics.classification_report(y_test, y_predicted, 
                                    target_names=dataset.target_names)) # classification result
cm = metrics.confusion_matrix(y_test, y_predicted) # confusion matrix
print(cm)

======= Start Evaluation of Random Forest =======
0 params - {'vect__ngram_range': (1, 2), 'vect__min_df': 2, 'vect__max_df': 0.98, 'clf__n_estimators': 4}; mean - 0.59; std - 0.02
1 params - {'vect__ngram_range': (1, 3), 'vect__min_df': 2, 'vect__max_df': 0.98, 'clf__n_estimators': 4}; mean - 0.59; std - 0.02
2 params - {'vect__ngram_range': (1, 2), 'vect__min_df': 3, 'vect__max_df': 0.98, 'clf__n_estimators': 4}; mean - 0.61; std - 0.01
3 params - {'vect__ngram_range': (1, 3), 'vect__min_df': 3, 'vect__max_df': 0.98, 'clf__n_estimators': 4}; mean - 0.61; std - 0.01
4 params - {'vect__ngram_range': (1, 2), 'vect__min_df': 4, 'vect__max_df': 0.98, 'clf__n_estimators': 4}; mean - 0.59; std - 0.03
5 params - {'vect__ngram_range': (1, 3), 'vect__min_df': 4, 'vect__max_df': 0.98, 'clf__n_estimators': 4}; mean - 0.61; std - 0.00
6 params - {'vect__ngram_range': (1, 2), 'vect__min_df': 2, 'vect__max_df': 0.95, 'clf__n_estimators': 4}; mean - 0.60; std - 0.01
7 params - {'vect__ngram_range': 

In [28]:
# Neural Network
print ('======= Start Evaluation of Neural Network =======')
pipeline = Pipeline([
        ('vect', TfidfVectorizer()), # vectorization
        ('clf', MLPClassifier()), # neural network
    ])

parameters = {
        'vect__ngram_range': [(1, 2), (1, 3)],
        'vect__min_df': [2, 3, 4],
        'vect__max_df': [0.98, 0.95, 0.92],
        'clf__alpha': [1e-5, 1e-4, 1e-3],
    }

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1)
grid_search.fit(docs_train, y_train)

n_candidates = len(grid_search.cv_results_['params'])
for i in range(n_candidates):
    print(i, 'params - %s; mean - %0.2f; std - %0.2f'
             % (grid_search.cv_results_['params'][i],
                grid_search.cv_results_['mean_test_score'][i],
                grid_search.cv_results_['std_test_score'][i]))

y_predicted = grid_search.predict(docs_test)
print(metrics.classification_report(y_test, y_predicted, 
                                    target_names=dataset.target_names)) # classification result
cm = metrics.confusion_matrix(y_test, y_predicted) # confusion matrix
print(cm)

======= Start Evaluation of Random Forest =======
0 params - {'vect__ngram_range': (1, 2), 'vect__min_df': 2, 'clf__alpha': 1e-05, 'vect__max_df': 0.98}; mean - 0.84; std - 0.00
1 params - {'vect__ngram_range': (1, 3), 'vect__min_df': 2, 'clf__alpha': 1e-05, 'vect__max_df': 0.98}; mean - 0.84; std - 0.00
2 params - {'vect__ngram_range': (1, 2), 'vect__min_df': 3, 'clf__alpha': 1e-05, 'vect__max_df': 0.98}; mean - 0.84; std - 0.00
3 params - {'vect__ngram_range': (1, 3), 'vect__min_df': 3, 'clf__alpha': 1e-05, 'vect__max_df': 0.98}; mean - 0.84; std - 0.01
4 params - {'vect__ngram_range': (1, 2), 'vect__min_df': 4, 'clf__alpha': 1e-05, 'vect__max_df': 0.98}; mean - 0.84; std - 0.00
5 params - {'vect__ngram_range': (1, 3), 'vect__min_df': 4, 'clf__alpha': 1e-05, 'vect__max_df': 0.98}; mean - 0.85; std - 0.01
6 params - {'vect__ngram_range': (1, 2), 'vect__min_df': 2, 'clf__alpha': 1e-05, 'vect__max_df': 0.95}; mean - 0.84; std - 0.01
7 params - {'vect__ngram_range': (1, 3), 'vect__min_df

*------------------------

## Problem 4 (20 points): Open Ended Question:  Finding the right plot

* Can you find a two dimensional plot in which the positive and negative reviews are separated?
    * This problem is hard since you will likely have thousands of features for review, and you will need to transform these thousands of features into just two numbers (so that you can make a 2D plot).
* Note, I was not able to find such a plot myself!
    * So, this problem is about **trying** but perhaps **not necessarily succeeding**!
* I tried two things, neither of which worked very well.
    * I first plotted the length of the review versus the number of features we compute that are in that review
    * Second I used Principle Component Analysis on a subset of the features.
* Can you do better than I did!?

# Report: communicate the results (20 points)

(1) (5 points) What data you collected?

(2) (5 points) Why this topic is interesting or important to you? (Motivations)

(3) (5 points) How did you analyse the data?

(4) (5 points) What did you find in the data?
(please include figures or tables in the report, but no source code)

# Slides (for 10 minutes of presentation) (20 points)


1. (5 points) Motivation about the data collection, why the topic is interesting to you. 

2. (10 points) Communicating Results (figure/table)

3. (5 points) Story telling (How all the parts (data, analysis, result) fit together as a story?)

*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What is the relationship between this topic and Business Intelligence?
    * How did you analyse the data?
    * What did you find in the data? 
    * What conjectures did you make and how did you support or disprove them using data?
    * Did you find anything suprising in the data?
    * What business decision do you think this data could help answer?  Why?

   (please include figures or tables in the report, **but no source code**)

*Please compress all the files into a single zipped file.*


** How to submit: **

        Send an email to rcpaffenroth@wpi.edu and wliu3@wpi.edu with the subject: "[DS501] Case study 3-TEAM NUMBER ???".